## Steps 1-3

In [1]:
import pandas as pd

In [2]:
fname_school_locations = 'schoollocations2019.csv'
fname_school_enrolments = 'allschoolsFTEenrolmentsFeb2021.csv'

#importing the datasets into pandas dataframes
school_locations = pd.read_csv(fname_school_locations, encoding= 'unicode_escape')
school_enrolments = pd.read_csv(fname_school_enrolments, encoding= 'unicode_escape')

#saving as csv files
# school_locations.to_csv('school_locations.csv')
# school_enrolments.to_csv('school_enrolments.csv')

#saving as json files
# school_locations.to_json('school_locations.json')
# school_enrolments.to_json('school_enrolments.json')

## Step 4

In [ ]:
#finding out number of columns in each dataset\n",
columns_locations = school_locations.columns
columns_enrolments = school_enrolments.columns
print('School Locations dataset has ' + str(len(columns_locations)) +  ' columns')
print('School Enrolments dataset has ' + str(len(columns_enrolments)) + ' columns')

## Step 5

In [ ]:
#finding out number of rows in each dataset\n",
print('School Locations dataset has ' + str(len(school_locations.index)) +  ' rows')
print('School Enrolments dataset has ' + str(len(school_enrolments.index)) + ' rows')

## Step 6: School Locations

In [ ]:
from pandas.core.dtypes.common import is_numeric_dtype

#for data set school_locations, create a dataframe with the following headers
headers = ['Field Name', 'Description', 'Pandas Data Type', 'Data Scale', 'Min Value', 'MAx Value', 'Unique Values', 'Missing Value Count']
school_location_table_data = pd.DataFrame(columns=headers)

for col in school_locations:
    row = []
    row.insert(0,col)
    row.insert(1,'?')
    row.insert(2,school_locations[col].dtype)
    if is_numeric_dtype(row[2]):
        row.insert(4,school_locations[col].min())
        row.insert(5,school_locations[col].max())
        row.insert(3,str(school_locations[col].min()) + ' - ' + str(school_locations[col].max()))
    else:
        row.insert(4,'N/A')
        row.insert(5,'N/A')
        row.insert(3,'N/A')
    row.insert(6,len(school_locations[col].unique()))
    row.insert(7,school_locations[col].isnull().sum())

    #create a series with this list\n",
    row_series = pd.Series(row, index = headers)
    school_location_table_data = school_location_table_data.append(row_series, ignore_index=True)
school_location_table_data

## Step 7: Scatter Plot
### Year 12 Male and Female Student Distribution Across Education Sectors and School Types

In [ ]:
import plotly.express as px
fig = px.scatter(school_enrolments, x='Year 12 Males', y='Year 12 Females', color='Education_Sector', symbol='School_Type', title='Year 12 Male and Female Student Distribution Across Education Sectors and School Types')
fig.show()

### Prep Student Distribution in Catholic Schools

In [ ]:
fig = px.scatter(school_enrolments.query("Education_Sector=='Catholic'"), x='Prep Males Total', y='Prep Females Total', size='Grand Total', color='School_Type', hover_name='School_Name', size_max=30, title = 'Prep Student Distribution in Catholic Schools')
fig.show()

### Total Student Count of Schools Across Education Sectors

In [ ]:
fig = px.scatter(school_enrolments, x='Grand Total', y='Education_Sector', color='School_Type', title='Total Student Count of Schools Across Education Sectors')
fig.show()

## Step 7: Pie Chart
### Student Distribution Across Education Sectors

In [ ]:
fig = px.pie(school_enrolments, values='Grand Total', names='Education_Sector', title='Student Distribution Across Education Sectors')
fig.show()

### Student Distribution Across School Types

In [ ]:
fig = px.pie(school_enrolments, values='Grand Total', names='School_Type', title='Student Distribution Across School Types')
fig.show()

### Number of Schools per LGA

In [ ]:
df=school_locations
df['Num_Schools_in_LGA']=df['LGA_ID'].map(df['LGA_ID'].value_counts())
df_unique_LGA = df.drop_duplicates(subset=['LGA_ID'])
df_unique_LGA.loc[df_unique_LGA['Num_Schools_in_LGA'] < 25, 'LGA_Name'] = 'Other LGA'
fig = px.pie(df_unique_LGA, values='Num_Schools_in_LGA', names='LGA_Name', title='Number of Schools per LGA')
fig.show()

### Distribution of Schools in Monash City Council Area, by Education Sector

In [ ]:
schools_in_Monash = school_locations.query("LGA_Name == 'Monash (C)'")
schools_in_Monash['Num_Education_Sector'] = schools_in_Monash['Education_Sector'].map(schools_in_Monash['Education_Sector'].value_counts())
fig = px.pie(schools_in_Monash.drop_duplicates(subset=['Education_Sector']), values='Num_Education_Sector', names='Education_Sector', title='Distribution of Schools in Monash City Council Area, by Education Sector')
fig.show()

### Distribution of Schools in Monash City Council Area, by School Type

In [ ]:
schools_in_Monash['Num_School_Type']=schools_in_Monash['School_Type'].map(schools_in_Monash['School_Type'].value_counts())
fig = px.pie(schools_in_Monash.drop_duplicates(subset=['School_Type']), values='Num_School_Type', names='School_Type', title='Distribution of Schools in Monash City Council Area, by School Type')
fig.show()

## Step 7: Histogram
### Total Student Distribution Across Education Sectors

In [ ]:
fig = px.histogram(school_enrolments, x='Grand Total', nbins=20, color='Education_Sector', title='Total Student Distribution Across Education Sectors')
fig.show()

### Number of Schools in an LGA

In [ ]:
fig = px.histogram(school_locations, x='LGA_Name', color='Education_Sector', title='Number of Schools in an LGA')
fig.show()

## Step 7: Maps
### Number of Schools in an LGA, Annotated on Map

In [ ]:
import plotly.graph_objects as go
fig = px.scatter_geo(df.drop_duplicates(subset=['LGA_ID']), lat='Y', lon='X', color='LGA_Name', hover_name='LGA_Name', projection='natural earth', size='Num_Schools_in_LGA', title='Number of Schools in an LGA')
fig.update_layout(geo = go.layout.Geo(resolution = 50, showframe = True, showcoastlines = True, landcolor = 'rgb(229, 229, 229)', countrycolor = 'white', projection_type = 'mercator', lonaxis_range= [ 141.07, 149.77 ], lataxis_range= [ -38.76, -34.74 ], domain = dict(x = [ 0, 1 ], y = [ 0, 1 ])),)
fig.show()

## Step 8: Pearson Correlation Coefficient on Heat Maps

In [ ]:
pearson_enrolments = school_enrolments.corr(method='pearson', min_periods=1)
heat_map_Pearson_enrolments = px.imshow(pearson_enrolments, text_auto=True)
heat_map_Pearson_enrolments.show()

## Step 8: Spearman Correlation Coefficient on Heat Maps

In [ ]:
spearman_enrolments = school_enrolments.corr(method='spearman', min_periods=1)
heat_map_spearman_enrolments = px.imshow(spearman_enrolments, text_auto=True)
heat_map_spearman_enrolments.show()

## Step 9: Folium Maps
### Schools across the State of Victoria, Displayed on a Map

In [ ]:
import folium
#import numpy as np\n",
#edu_sectors = school_locations['Education_Sector'].unique()\n",

edu_sectors_dict = {'Government': 'green', 'Catholic': 'red', 'Independent': 'blue'}
school_types_dict = {'Primary': 'child', 'Secondary':'graduation-cap', 'Pri/Sec': 'university', 'Special': 'universal-access'}

m = folium.Map(location=[school_locations['Y'].mean(axis=0), school_locations['X'].mean(axis=0)], zoom_start=12)
for i in range(0,len(school_locations['Y'])):
    popup = school_locations.loc[i,'School_Name'] + '\n' + school_locations.loc[i,'School_Type'] + '(' + school_locations.loc[i,'Education_Sector'] + ')' + '\n' + school_locations.loc[i,'Address_Town'] + ',' + school_locations.loc[i,'LGA_Name']
    folium.Marker([school_locations.loc[i,'Y'], school_locations.loc[i,'X']], popup=popup, tooltip=school_locations.loc[i,'School_Name'], icon=folium.Icon(color=edu_sectors_dict.get(school_locations.loc[i,'Education_Sector']), icon=school_types_dict.get(school_locations.loc[i,'School_Type']), prefix = 'fa')).add_to(m)
m

## Step 10: Visualising School Locations with Pandas Profiling

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(school_locations, title='Pandas Profiling Report - School Locations', explorative=True)
profile.to_notebook_iframe()

## Step 10: Visualising School Enrolments with Pandas Profiling

In [ ]:
profile = ProfileReport(school_enrolments, title='Pandas Profiling Report - School Enrolments', explorative=True)
profile.to_notebook_iframe()